In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Kidney_Disease_Classification\\research'

In [3]:
cd ..

e:\Projects\Kidney_Disease_Classification


In [4]:
%pwd

'e:\\Projects\\Kidney_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *


In [7]:
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        try:
            config = self.config.prepare_base_model
        except KeyError as e:
            print(f"KeyError: '{e.args[0]}' not found in the loaded configuration.")
            print("Loaded Configuration:", self.config)
            raise

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
from cnnClassifier import config


print(dir(config))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [19]:
print(type(config))  # To check the type of the `config` attribute



<class '__main__.ConfigurationManager'>


In [20]:
config

In [16]:
config.get_prepare_base_model_config() # To check the type of the returned object

[2024-05-13 21:18:01,481: INFO: common: created directory at: artifacts/prepare_base_model]


PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)

In [15]:
config_data = {
    'artifacts_root': 'artifacts', 
    'data_ingestion': {
        'root_dir': 'artifacts/data_ingestion', 
        'source_URL': 'https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing', 
        'local_data_file': 'artifacts/data_ingestion/data.zip', 
        'unzip_dir': 'artifacts/data_ingestion'
    }
}


In [14]:
print(config_data['artifacts_root'])  # prints 'artifacts'

print(config_data['data_ingestion']['source_URL'])  # prints the source URL


artifacts
https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing


In [12]:
import yaml

with open('config\config.yaml', 'r') as file:
    config_data = yaml.safe_load(file)


In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e


[2024-05-13 21:15:38,422: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-13 21:15:38,522: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-13 21:15:38,558: INFO: common: created directory at: artifacts]
[2024-05-13 21:15:38,563: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-05-13 21:15:44,504: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64